In [4]:
import pandas as pd

# CSVデータの読み込み
data = pd.read_csv("race_data_1_6000/race_data_1_10.csv")

# DataFrameの作成
df = pd.DataFrame(data)

# Codeが四文字のデータを削除
df = df[~df['Code'].astype(str).str.match(r'^\d{4}$')]

# Horse Weightを変形
df[['Weight', 'Weight Change']] = df['Horse Weight'].str.extract(r'(\d+)\s*\(\s*([+-]?\d+)\s*\)')
df['Weight'] = df['Weight'].fillna(0)
df['Weight'] = df['Weight'].astype(int)
df['Weight Change'] = df['Weight Change'].fillna(0)
df['Weight Change'] = df['Weight Change'].astype(int)
df.drop(columns=['Horse Weight'], inplace=True)

In [5]:
# 「Sex/Age」列を「Sex」と「Age」に分ける
df['Sex'] = df['Sex/Age'].str[0].map({'牡': 0, '牝': 1}).astype('Int64')  
df['Age'] = pd.to_numeric(df['Sex/Age'].str[1:], errors='coerce').astype('Int64') # errors='coerce'でエラーをNaNに変換
df.drop(columns=['Sex/Age'], inplace=True)

In [6]:
# 時間を分表記から秒表記に変更
def convert_to_second(time_str):
    if isinstance(time_str, str):
        minutes, seconds = map(float, time_str.split(':'))
        return minutes * 60 + seconds
    return time_str  
df['Time_x'] = df['Time_x'].apply(convert_to_second)

In [7]:
# Distanceを分離
df['Ground'] = df['Distance'].str[0]
df['Distance'] = df['Distance'].str[1:]

# Ground, Condition, Weatherをダミー変数に変換
dummy = pd.get_dummies(df[['Ground', 'Condition', 'Weather']])
dummy

# データフレームを結合し、要らないデータを破棄
df = pd.concat([df, dummy], axis=1)
df.drop(columns=['Ground', 'Condition', 'Weather'], inplace=True)
df

# first_chars = df['Condition'].dropna().apply(lambda x: str(x)[0])  # NaNを除外し、先頭文字を取得
# unique_first_chars = first_chars.unique()  # ユニークな文字を取得

# # 結果を表示
# print("0インデックス目のユニークな文字:", list(unique_first_chars))

KeyError: 'Distance'

In [28]:
df.to_csv('updated_horse_data.csv', index=False)